# Weather image recognition

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

import math
import numpy as np
import matplotlib.pyplot as plt

print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.10.1


### Model hyperparameters

In [4]:
batch_size = 32
epochs = 8

### Data paths

In [5]:
local_folder = './dataset'
train_dir = local_folder + '/train'
valid_dir = local_folder + '/valid'
test_dir = local_folder + '/test'

### Load datasets

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip=True, vertical_flip=True, rotation_range=10)

target_image_size = (256, 256)

train_dataset = datagen.flow_from_directory(train_dir,
                                                  target_size = target_image_size,
                                                  batch_size = batch_size,
                                                  class_mode="categorical")

test_dataset = datagen.flow_from_directory(test_dir,
                                                target_size = target_image_size,
                                                batch_size = batch_size,
                                                class_mode="categorical")

valid_dataset = datagen.flow_from_directory(valid_dir,
                                                  target_size = target_image_size,
                                                  batch_size = batch_size,
                                                  class_mode="categorical")

Found 5484 images belonging to 11 classes.
Found 696 images belonging to 11 classes.
Found 682 images belonging to 11 classes.


### Number of images each dataset contains

In [ ]:
num_train_examples = 5484
num_test_examples = 696
num_valid_examples = 682

## Convolutional Neural Network model

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape = (256, 256, 3)))
model.add(tf.keras.layers.MaxPooling2D(strides = 2))
model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(strides = 2))
model.add(tf.keras.layers.Flatten(input_shape=(7, 7, 128)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(11, activation='softmax'))

In [7]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['acc'])

In [8]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 524288)            0         
                                                                 
 dense (Dense)               (None, 128)               6

## Model training

In [ ]:
history = model.fit(train_dataset, epochs=epochs, validation_data=valid_dataset)

Epoch 1/8
172/172 [==============================] - 75s 437ms/step - loss: 0.7141 - acc: 0.7613 - val_loss: 1.0744 - val_acc: 0.6540
Epoch 2/8
172/172 [==============================] - 75s 434ms/step - loss: 0.6653 - acc: 0.7774 - val_loss: 1.0324 - val_acc: 0.6760
Epoch 3/8
172/172 [==============================] - 75s 435ms/step - loss: 0.6025 - acc: 0.7965 - val_loss: 1.0682 - val_acc: 0.6848
Epoch 4/8
172/172 [==============================] - 76s 439ms/step - loss: 0.5844 - acc: 0.7989 - val_loss: 0.9445 - val_acc: 0.6994
Epoch 5/8
172/172 [==============================] - 75s 437ms/step - loss: 0.5413 - acc: 0.8208 - val_loss: 0.9730 - val_acc: 0.7053
Epoch 6/8
172/172 [==============================] - 75s 434ms/step - loss: 0.5279 - acc: 0.8187 - val_loss: 1.0186 - val_acc: 0.7009
Epoch 7/8
172/172 [==============================] - 74s 432ms/step - loss: 0.4926 - acc: 0.8350 - val_loss: 1.0169 - val_acc: 0.6716
Epoch 8/8
172/172 [==============================] - 75s 435ms

## Train and test datasets accuracy

In [ ]:
train_loss, train_accuracy = model.evaluate(train_dataset, steps=math.ceil(num_train_examples / batch_size))
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples / batch_size))

print('Train accuracy: {:.2f}%'.format(100*train_accuracy))
print('Test accuracy: {:.2f}%'.format(100*test_accuracy))

22/22 [==============================] - 9s 413ms/step - loss: 1.0744 - acc: 0.6940
Train accuracy: 84.72%
Test accuracy: 69.40%
